In [1]:
import json, random
import os
import numpy as np
from PIL import Image
import matplotlib.pylab as plt
import sys
from keras.utils import to_categorical
from keras import models, layers
from keras import optimizers
import tensorflow as tf
import shutil

In [2]:
image_path = "../dataset/train/image/"
image_files = os.listdir(image_path)

Korean_id = []
Korean_text = []

for f in image_files:
    filename = str(f) 
    filename_list = filename.split('.')
    filename = filename_list[0]          # IMG_OCR_53_4PR_18338_29
    json_path = filename.split('_')[:5]
    json_path = '_'.join(json_path)      # IMG_OCR_53_4PR_18338
    id = filename.split('_')[5:] 
    id = (int)(id[0])                    # 29
    
    with open('../dataset/train/json/' + json_path + '.json', 'r', encoding='utf-8') as f:
        file = json.load(f)
    bbox = file['bbox']
    
    for index, box in enumerate(bbox):
        if id == file['bbox'][index]['id']:
            text = file['bbox'][index]['data']
            id = file['bbox'][index]['id']
    Korean_id.append(filename)          # ['IMG_OCR_53_4PR_09180_2']
    Korean_text.append(text)            # ['광주광역시']

['IMG_OCR_53_4PR_09180_2']
['광주광역시']
['IMG_OCR_53_4PR_09180_2', 'IMG_OCR_53_4PR_09180_30']
['광주광역시', '박길서']
['IMG_OCR_53_4PR_09180_2', 'IMG_OCR_53_4PR_09180_30', 'IMG_OCR_53_4PR_09184_33']
['광주광역시', '박길서', '인태장']
['IMG_OCR_53_4PR_09180_2', 'IMG_OCR_53_4PR_09180_30', 'IMG_OCR_53_4PR_09184_33', 'IMG_OCR_53_4PR_09187_15']
['광주광역시', '박길서', '인태장', '동정동']
['IMG_OCR_53_4PR_09180_2', 'IMG_OCR_53_4PR_09180_30', 'IMG_OCR_53_4PR_09184_33', 'IMG_OCR_53_4PR_09187_15', 'IMG_OCR_53_4PR_09187_33']
['광주광역시', '박길서', '인태장', '동정동', '우부상']
['IMG_OCR_53_4PR_09180_2', 'IMG_OCR_53_4PR_09180_30', 'IMG_OCR_53_4PR_09184_33', 'IMG_OCR_53_4PR_09187_15', 'IMG_OCR_53_4PR_09187_33', 'IMG_OCR_53_4PR_09189_17']
['광주광역시', '박길서', '인태장', '동정동', '우부상', '대마면']
['IMG_OCR_53_4PR_09180_2', 'IMG_OCR_53_4PR_09180_30', 'IMG_OCR_53_4PR_09184_33', 'IMG_OCR_53_4PR_09187_15', 'IMG_OCR_53_4PR_09187_33', 'IMG_OCR_53_4PR_09189_17', 'IMG_OCR_53_4PR_09189_27']
['광주광역시', '박길서', '인태장', '동정동', '우부상', '대마면', 'ㅇ_ㅇ']
['IMG_OCR_53_4PR_09180_2', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
print(len(Korean_id))
print(len(Korean_text))

7922
7922


In [4]:
Korean_train_id = Korean_id[0:7200]
Korean_train_text = Korean_text[0:7200]

Korean_test_id = Korean_id[7200:]
Korean_test_text = Korean_text[7200:]

word = list(set(Korean_text))

del(Korean_id)
del(Korean_text)

In [5]:
x_train = np.zeros((7200, 32, 32, 3), 'uint8')
x_test = np.zeros((722, 32, 32, 3), 'uint8')

for i, ID in enumerate(Korean_train_id) :
    Image_addr = "../dataset/train/image/" + str(ID) + ".png"
    Korean_Image = Image.open(Image_addr)
    Korean_Image = Korean_Image.resize((32, 32))
    Korean_Image_Array = np.array(Korean_Image, 'uint8')
    x_train[i] = Korean_Image_Array

for i, ID in enumerate(Korean_test_id) :
    Image_addr = "../dataset/train/image/" + str(ID) + ".png"
    Korean_Image = Image.open(Image_addr)
    Korean_Image = Korean_Image.resize((32, 32))
    Korean_Image_Array = np.array(Korean_Image, 'uint8')
    x_test[i] = Korean_Image_Array

In [6]:
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

del(Korean_train_id)
del(Korean_test_id)

In [7]:
word_to_index = {word : index for index, word in enumerate(word)}
index_to_word = {index : word for index, word in enumerate(word)}

label_train = []
label_test = []

for words in Korean_train_text : 
    if word_to_index.get(words) is not None: 
        label_train.extend([word_to_index[words]])
        
for words in Korean_test_text : 
    if word_to_index.get(words) is not None : 
        label_test.extend([word_to_index[words]])

del(Korean_train_text)
del(Korean_test_text)

In [8]:
print(len(label_train))
print(len(label_test))
print(len(x_train))
print(len(x_test))

7200
722
7200
722


In [9]:
# status initialize
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 

# out of memory
with tf.Graph().as_default():
  gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)

In [10]:
label_train = to_categorical(label_train, num_classes=7922)
label_test = to_categorical(label_test, num_classes=7922)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(7922, activation='softmax')
])

model.compile(optimizer=optimizers.RMSprop(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])

with tf.device("/device:GPU:0"):
    model.fit(x_train, label_train, epochs = 100, batch_size = 128, validation_split=0.3)

test_loss, test_acc = model.evaluate(x_test, label_test)
print('test_loss:      ',test_loss)
print('test_accuracy:  ',test_acc)

Epoch 1/100
40/40 [==============================] - 13s 107ms/step - loss: 1799.1877 - accuracy: 0.0062 - val_loss: 8.7699 - val_accuracy: 0.0042
Epoch 2/100
40/40 [==============================] - 3s 68ms/step - loss: 9.9082 - accuracy: 0.0050 - val_loss: 9.0064 - val_accuracy: 0.0046
Epoch 3/100
40/40 [==============================] - 3s 69ms/step - loss: 8.4734 - accuracy: 0.0063 - val_loss: 9.2799 - val_accuracy: 0.0046
Epoch 4/100
40/40 [==============================] - 3s 69ms/step - loss: 7.8963 - accuracy: 0.0056 - val_loss: 9.6344 - val_accuracy: 0.0056
Epoch 5/100
40/40 [==============================] - 3s 69ms/step - loss: 7.8479 - accuracy: 0.0054 - val_loss: 9.8639 - val_accuracy: 0.0083
Epoch 6/100
40/40 [==============================] - 3s 69ms/step - loss: 13.8826 - accuracy: 0.0046 - val_loss: 10.0039 - val_accuracy: 0.0037
Epoch 7/100
40/40 [==============================] - 3s 68ms/step - loss: 7.8155 - accuracy: 0.0056 - val_loss: 9.8327 - val_accuracy: 0.0046

40/40 [==============================] - 3s 74ms/step - loss: 7.7226 - accuracy: 0.0063 - val_loss: 12.6236 - val_accuracy: 0.0083
Epoch 59/100
40/40 [==============================] - 3s 71ms/step - loss: 7.7235 - accuracy: 0.0069 - val_loss: 12.6315 - val_accuracy: 0.0083
Epoch 60/100
40/40 [==============================] - 3s 68ms/step - loss: 7.7229 - accuracy: 0.0065 - val_loss: 12.6393 - val_accuracy: 0.0083
Epoch 61/100
40/40 [==============================] - 3s 69ms/step - loss: 7.7229 - accuracy: 0.0058 - val_loss: 12.6470 - val_accuracy: 0.0083
Epoch 62/100
40/40 [==============================] - 3s 70ms/step - loss: 7.7229 - accuracy: 0.0079 - val_loss: 12.6537 - val_accuracy: 0.0056
Epoch 63/100
40/40 [==============================] - 3s 69ms/step - loss: 7.7226 - accuracy: 0.0073 - val_loss: 12.6596 - val_accuracy: 0.0056
Epoch 64/100
40/40 [==============================] - 3s 68ms/step - loss: 7.7231 - accuracy: 0.0075 - val_loss: 12.6656 - val_accuracy: 0.0056
Epoch

In [11]:
model.save('../model/Korean_CNN_model_50000.h5')

with open("../model/Korean_CNN_model_50000.json", "w") as json_file:
   json.dump(index_to_word, json_file)